In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

In [2]:
%%cu
#include <math.h>
#include <chrono>
#include <iostream> 
#include <sys/time.h>
#include <stdio.h>

double f1(double x) { return x * x + x + 10; }

double metoda_prostokatow(int n, float xp, float xk) {
    float calka = 0;
    float h = (xk - xp) / (float)n;
    int i = 1;

    for (i = 1; i <= n; i++) {
        calka += f1(xp + i*h)*h;
    }

    return calka; 
}

double metoda_trapezow(int n, float xp, float xk) {
    float calka = 0;
    float h = (xk - xp) / (float)n;
    int i = 1;

    for (i = 1; i < n; i++) {
        calka += f1(xp + i * h);
    }
    calka += f1(xp) / 2;
    calka += f1(xk) / 2;
    calka *= h;

    return calka;
}

double metoda_simpsona(int n, float xp, float xk) {
    float calka = 0;
    float h = (xk - xp) / (float)n;
    float s = 0;
    float x;
    int i = 1;
    
    for (i = 1; i < n; i++) {
      x = xp + i * h;
      s += f1(x - h / 2);
      calka += f1(x);
    }
    s += f1(xk - h / 2);
    calka = (h / 6) * (f1(xp) + f1(xk) + 2 * calka + 4 * s);

    return calka;
}

int main() {
    float xp, xk;
    int n = 1000;
    int k;

    // przedzialy
    xp = 1;
    xk = 10000000;

  //printf("n = %d | xp = %d | xk = %d\n",n ,xp, xk);
for(k = 0; k < 5; k++) {
    n *= 10;
printf("n = %d\n",n);

//  metoda prostokatow
  float w1;
    struct timeval begin, end;
    gettimeofday(&begin, 0);

  w1 = metoda_prostokatow(n, xp, xk);

    gettimeofday(&end, 0);
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;

  printf("metoda prostokatow czas = %.8f\n",elapsed);

//  metoda prostokatow

//  metoda trapezow
  float w2;
  struct timeval begin2, end2;
    gettimeofday(&begin2, 0);

  w2 = metoda_trapezow(n, xp, xk);

  gettimeofday(&end2, 0);
    long seconds2 = end2.tv_sec - begin2.tv_sec;
    long microseconds2 = end2.tv_usec - begin2.tv_usec;
    double elapsed2 = seconds2 + microseconds2*1e-6;

  printf("metoda trapezow czas = %.8f\n",elapsed);
  
//  metoda trapezow

//  metoda simpsona
  float w3;
  struct timeval begin3, end3;
    gettimeofday(&begin3, 0);

  w3 = metoda_simpsona(n, xp, xk);

  gettimeofday(&end3, 0);
    long seconds3 = end3.tv_sec - begin3.tv_sec;
    long microseconds3 = end3.tv_usec - begin3.tv_usec;
    double elapsed3 = seconds3 + microseconds3*1e-6;

  printf("metoda simpsona czas = %.8f\n",elapsed3);
  
//  metoda simpsona
}


}

n = 10000
metoda prostokatow czas = 0.00017400
metoda trapezow czas = 0.00017400
metoda simpsona czas = 0.00030600
n = 100000
metoda prostokatow czas = 0.00177600
metoda trapezow czas = 0.00177600
metoda simpsona czas = 0.00306000
n = 1000000
metoda prostokatow czas = 0.01943900
metoda trapezow czas = 0.01943900
metoda simpsona czas = 0.03056900
n = 10000000
metoda prostokatow czas = 0.17608900
metoda trapezow czas = 0.17608900
metoda simpsona czas = 0.31006300
n = 100000000
metoda prostokatow czas = 1.76552900
metoda trapezow czas = 1.76552900
metoda simpsona czas = 3.05540400



In [4]:
%%cu
#include <sys/time.h>
#include <stdio.h>
#include <chrono>
#include <iostream>
#include <math.h>
using namespace std;

__device__ float fg(float x) { return x * x + x + 10; }


__global__ void watek_prostokaty( float xp, float h, int n, float* tab) {
    
  int tid = blockIdx.x + 1;

      if(tid < n){
      
      tab[tid - 1] = fg(xp + tid * h) * h;
      }

  
}

__global__ void watek_trapezy( float xp, float h, int n, float* tab) {
    
		int tid = blockIdx.x + 1;

      if(tid < n){
      
      tab[tid-1] = fg(xp + tid * h);
      }
  
}

__global__ void watek_simpson( float xp, float h, int n, float* tab, float* tab2 ) {
    
		int tid = blockIdx.x + 1;
  
      if(tid < n + 1){
		tab2[tid - 1] = xp + tid * h;
		tab[tid - 1] = fg(xp + tid * h);
      }
  
}

float metoda_prostokatow( float xp, float xk, int n) {
    
    //cudaError_t status;
    size_t size = n * sizeof(float);

    float* dev;
    float* host = (float*)malloc(size);
    

    float suma = 0;
    cudaMalloc((void**) &dev,size);
    float h = (xk - xp) / (float)n;
    
    watek_prostokaty<<<n,1>>> (xp, h, n, dev);
    cudaMemcpy(host, dev, sizeof(float) * n, cudaMemcpyDeviceToHost);
    for(int i = 0; i < n; i++){
        suma += host[i];
    }
    free(host);
    cudaFree(dev);


    return suma;

}

 float f1(float x) { return x * x + x + 10; }

float metoda_trapezow( float xp, float xk, int n) {
    
    //cudaError_t status;
    size_t size = n * sizeof(float);

    float* dev;
    float* host = (float*)malloc(size);
    

    float suma = 0;
    cudaMalloc((void**) &dev,size);
    float h = (xk - xp) / (float)n;
    
    watek_trapezy<<<n,1>>> (xp, h, n, dev);
    cudaMemcpy(host, dev, sizeof(float) * n, cudaMemcpyDeviceToHost);
    
    for(int i = 0; i < n; i++){
        suma += host[i];
    }
    
    free(host);
    cudaFree(dev);

    return h*((f1(xp) / 2) + suma + (f1(xk) / 2));

}

float metoda_simpsona( float xp, float xk, int n) {
    
    //cudaError_t status;
    size_t size = n * sizeof(float);

    float* dev;
    float* host = (float*)malloc(size);
    float* dev2;
    float* host2 = (float*)malloc(size);
    

    
    float suma1 = 0 ,suma2 = 0;
    float t[n];
    cudaMalloc((void**) &dev,size);
    cudaMalloc((void**) &dev2,size);
    float h = (xk - xp) / (float)n;
    
    watek_simpson<<<n,1>>> (xp, h, n, dev, dev2);
    cudaMemcpy(host, dev, sizeof(float)*n, cudaMemcpyDeviceToHost);
    cudaMemcpy(host2, dev2, sizeof(float)*n, cudaMemcpyDeviceToHost);
    
    for(int i = 0; i < n; i++){
        t[i] = f1((host2[i + 1] +host2[i]) / 2);
    }

    for(int i = 0 ; i < n; i++){
            suma1 += t[i];
        }

    suma1 *= 4;
    for(int i = 1 ; i < n; i++){
        suma2 += host[i];
    }

    suma2 *= 2;
    free(host);
    cudaFree(dev);
    free(host2);
    cudaFree(dev2);
    
    return (f1(xp) + host[n] + suma1 + suma2) * (h / 6);

}

int main() {
    
    float xp, xk;
    int n = 10000;
    int k;

    // przedzialy
    xp = 1;
    xk = 10000000;

    //printf("n = %d\n",n);
    cout<<"n = "<<n<<endl;
        //prostokaty
        auto start1 = chrono::steady_clock::now();
        float w1 = metoda_prostokatow(xp, xk, n);
        auto end1 = chrono::steady_clock::now();
        auto diff1 = end1 - start1;
        cout<<"metoda prostokatow czas = "<<(chrono::duration <double, milli> (diff1).count()) / 1000<<endl;
        //prostokaty
        //trapezy
        auto start2 = chrono::steady_clock::now();
        float w2 = metoda_trapezow(xp, xk, n);
        auto end2 = chrono::steady_clock::now();
        auto diff2 = end2 - start2;
        cout<<"metoda trapezow czas = "<<(chrono::duration <double, milli> (diff2).count()) / 1000<<endl;
        //trapezy
        //simpson
        auto start3 = chrono::steady_clock::now();
        float w3 = metoda_simpsona(xp, xk, n);
        auto end3 = chrono::steady_clock::now();
        auto diff3 = end3 - start3;
        cout<<"metoda simpsona czas = "<<(chrono::duration <double, milli> (diff3).count()) / 1000<<endl;
        //simpson




    return 0;
}

n = 10000
metoda prostokatow czas = 0.375104
metoda trapezow czas = 0.000372843
metoda simpsona czas = 0.000515513

